# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [207]:
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [208]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [209]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [210]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [211]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

In [230]:
#explore the XML root
#for child in root:
#    print (child.tag, child.attrib)

In [213]:
#practise the XML tools
root[0][1].text

'1214489'

# 1. Lowest mortality rates

In [214]:
#make a blank dictionary
dict = {}

#loop through the elements to find country and the child 'infant mortality'
for baby in document.iterfind('country'):
    country = baby.find('name')
    mortality = baby.find('infant_mortality')

#dump blanks
    if (country) != None:
        if (mortality) != None:
#build the dictionary by setting the key to the value
            dict[country.text] = float(mortality.text)
        else:
            dict[country.text] = np.NaN

# Convert to data frame from dictionary
dfPais = pd.DataFrame.from_dict(dict, orient='index')

# Set Column to Infant Mortality Rate
dfPais.columns = ['Baby Death Rate per 1000']
dfPais.sort_values(by='Baby Death Rate per 1000', ascending=True).head(10)

,Baby Death Rate per 1000
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


# 2. 10 Cities with the largest population

In [215]:
#make a blank list
pop_list = []

#loop through the elements to find cities and provinces as they both have populations
for element in document.iterfind('country'):
    cities = element.findall('city')
    provinces = element.findall('province')

#access the population in each province and corresponding city
#add them to the city list
    for province in provinces:
        cities += province.findall('city')
#get the individual city name and the most recent population value
    for city in cities:
        cityName = city.find('name').text
        population = city.find('.//population[last()]')
        if population != None:
            cityPop = int(population.text)
        cityList = (cityName,cityPop)
#populate the list
        pop_list.append(cityList)
#create a dataframe

df = pd.DataFrame.from_records(data=pop_list,columns=['city','pop'])
df
#print the dataframe to show the largest 10 by sorting it
df.sort_values('pop',ascending=False).head(10)

,city,pop
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


# 3.0 largest ethnic groups

In [216]:
#make a blank list
ethpop_list = []

#loop through the elements to find single country
for element in document.iterfind('country'):
    ctry = element.find('name').text
#within the country, get the ethnicity detail  
    for eth in element.iterfind('ethnicgroup'):
        ethnicName = eth.text
        ethnicityPctg = float(eth.attrib['percentage'])
        ethpop_list.append([ctry,ethnicName,ethnicityPctg])
#populate the list
#create a dataframe

df = pd.DataFrame.from_records(data=ethpop_list,columns=['country','ethnicGroup','pop%'])
df
#print the dataframe to show the largest 10 by sorting it
df.sort_values('pop%',ascending=True).head(3)

,country,ethnicGroup,pop%
614,Mozambique,European,0.06
615,Mozambique,Indian,0.08
56,Slovakia,Polish,0.10


In [217]:
#make a blank dictionary
ctry_list = {}

#loop the document for country and population
for country in document.iterfind('country'):
    ctry = country.find('name')
    population = country.find('.//population[last()]')
# dictionary of counrty and population

    ctry_list[ctry.text] = int(population.text)


# Creates dataframe
ctry_df = pd.DataFrame.from_dict(ctry_list, orient='index')
ctry_df.reset_index(drop=False, inplace=True)
ctry_df.columns = ['country', 'Population']


In [218]:
#merge the dataframes, creat the ethnicPopulation column, sum them by group and sort for highest
ethnicDF=ctry_df.merge(df,on='country')
ethnicDF['ethnicPop']=ethnicDF['pop%']*ethnicDF['Population']/100
ethnicities=ethnicDF.groupby('ethnicGroup').sum()
ethnicities.sort_values('ethnicPop',ascending=False).head(10)

,Population,pop%,ethnicPop
ethnicGroup,,,
Han Chinese,1360720000,91.50,1.245059e+09
Indo-Aryan,1210854977,72.00,8.718156e+08
European,1157295639,970.82,4.948722e+08
African,975352746,1868.55,3.183251e+08
Dravidian,1210854977,25.00,3.027137e+08
Mestizo,279743964,870.70,1.577344e+08
Bengali,149772364,98.00,1.467769e+08
Russian,322438406,224.10,1.318570e+08
Japanese,127298000,99.40,1.265342e+08


## 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [219]:
#make a blank dictionary
ctrycode_list = {}

#loop the document for country and population
for country in document.iterfind('country'):
    ctry = country.find('name')
    ctrycode = country.attrib['car_code']
# dictionary of counrty and population

    ctrycode_list[ctrycode] = ctry.text


# Creates dataframe
ctry_df = pd.DataFrame.from_dict(ctrycode_list, orient='index')
ctry_df.reset_index(drop=False, inplace=True)
ctry_df.columns = ['country', 'ctry code']
ctry_df.head()


,country,ctry code
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro


In [229]:
#make a blank list
water_list = []

#loop through the countries to find cities and rivers
for element in document.iterfind('river'):    
    riverName = element.find('name').text
    riverSize = element.find('length')
    if riverSize != None:
        
        riverLength = float(riverSize.text)
        for riverCC in element.attrib['country'].split():
            riverCC = ctrycode_list[riverCC]
    waterList = (riverCC,riverName,riverLength)
#populate the list
    water_list.append(waterList)
#create a dataframe

dfWater = pd.DataFrame.from_records(data=water_list,columns=['cc','waterBody','length'])

#print the dataframe sorted for longest river
dfWater.sort_values(by='length',ascending = False).head(10)


,cc,waterBody,length
174,Peru,Amazonas,6448.0
137,China,Jangtse,6380.0
136,China,Hwangho,4845.0
123,Russia,Lena,4400.0
205,Zaire,Zaire,4374.0
138,Vietnam,Mekong,4350.0
115,China,Irtysch,4248.0
186,Guinea,Niger,4184.0
160,United States,Missouri,4130.0
119,Russia,Jenissej,4092.0


# The longest river is the amazon river and 'originates' in Peru

In [228]:
lake_list=[]
#loop through the countries to find lakes and area of the lake
for element in document.iterfind('lake'):    
    lakeName = element.find('name').text
    lakeSize = element.find('area')
    if lakeSize != None:
        
        lakeArea = float(lakeSize.text)
        for lakeCC in element.attrib['country'].split():
            lakeCC = ctrycode_list[lakeCC]
    lakeList = (lakeCC,lakeName,lakeArea)
#populate the list
    lake_list.append(lakeList)
#create a dataframe

dfLake = pd.DataFrame.from_records(data=lake_list,columns=['cc','waterBody','area'])

#print the dataframe sorted for largest area lake
dfLake.sort_values(by='area',ascending = False).head(10)

,cc,waterBody,area
54,Turkmenistan,Caspian Sea,386400.0
109,United States,Lake Superior,82103.0
81,Uganda,Lake Victoria,68870.0
106,United States,Lake Huron,59600.0
108,United States,Lake Michigan,57800.0
47,West Bank,Dead Sea,41650.0
83,Tanzania,Lake Tanganjika,32893.0
98,Canada,Great Bear Lake,31792.0
43,Russia,Ozero Baikal,31492.0
89,Tanzania,Lake Malawi,29600.0


# The largest lake is the caspian sea in turkmenistan...largest fresh water is Lake Superior, equally owned by Canada

In [227]:
airport_list=[]
#loop through the countries to find airports and elevation
for element in document.iterfind('airport'):    
    airName = element.find('name').text
    airElevation = element.find('elevation')
    if airElevation != None:
        elevation = airElevation.text
        for airCC in element.attrib['country'].split():
            airCC = ctrycode_list[airCC]
    airList = (airCC,airName,elevation)
#populate the list
    airport_list.append(airList)
#create a dataframe

dfAirText = pd.DataFrame.from_records(data=airport_list,columns=['cc','airportName','elevation'])
#convert the data from text to numeric
dfAir=dfAirText.apply(pd.to_numeric, errors='ignore')
#print the dataframe sorted for largest elevation
dfAir.sort_values(by='elevation',ascending = False).head(10)



,cc,airportName,elevation
80,Bolivia,El Alto Intl,4063.0
219,China,Lhasa-Gonggar,4005.0
241,China,Yushu Batang,3963.0
813,Peru,Juliaca,3827.0
815,Peru,Teniente Alejandro Velasco Astete Intl,3311.0
82,Bolivia,Juana Azurduy De Padilla,2905.0
334,Ecuador,Mariscal Sucre Intl,2813.0
805,Peru,Coronel Fap Alfredo Mendivil Duarte,2719.0
807,Peru,Mayor General FAP Armando Revoredo Iglesias Ai...,2677.0
692,Mexico,Licenciado Adolfo Lopez Mateos Intl,2581.0


# The highest airport in the worls is 'El Alto Intl' meaning 'the height' in Bolivia